In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

## Visit the NASA mars news site

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# time.sleep(1)  # Optional delay for loading the page

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
# browser.quit()

In [6]:
# Use beautiful soup to find the first news title parent element
listTextLabelElem = news_soup.find('div', class_='listTextLabel')

In [7]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = listTextLabelElem.find('a').get_text()
news_title

"NASA's Next Mars Mission to Investigate Interior of Red Planet"

In [8]:
# Use the parent element to find the paragraph text
news_p = listTextLabelElem.find('p').get_text()
news_p

"Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."

## JPL Space Images Featured Image

In [4]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [5]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [6]:
# Find the more info button and click that
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [7]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [8]:
# find the relative image url
img_url_rel = img_soup.find('figure', class_='lede').find('img')['src']
img_url_rel

'/spaceimages/images/largesize/PIA17011_hires.jpg'

In [9]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17011_hires.jpg'

## Mars Weather

In [10]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [11]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [12]:
# First, find a tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [13]:
# Next, search within the tweet for the p tag containing the tweet text
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'Welcome Spring! (to @MarsCuriosity’s home in the southern hemisphere of Mars). May 22 at 1431 UTC Mars reached a solar longitude of 180° in its trip around the sunpic.twitter.com/bzbkEwvfE4'

In [14]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [16]:
# browser.find_by_css("a.product-item")[i].click()

NameError: name 'i' is not defined

In [15]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()
    

WebDriverException: Message: unknown error: Element <a href="/search/map/Mars/Viking/cerberus_enhanced" class="itemLink product-item">...</a> is not clickable at point (53, 228). Other element would receive the click: <div class="item">...</div>
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.11.6 x86_64)


In [16]:
hemisphere_image_urls

[]

## Mars Facts

In [17]:
import pandas as pd
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [20]:
table = df.to_html('index.html')


In [21]:
browser.quit()